In [172]:
import pandas as pd
from IPython.display import display
import json

In [173]:
# open raw data
file_path = "/Users/FranklinZhao/TensorFlowProjects/ImageBasedSneakerPrediction/data/rawoutput.csv"
df_ori= pd.read_csv(filepath_or_buffer=file_path, sep='\t', index_col=0)
df = df_ori.copy()

In [174]:
# remove unnecessary data
cols_to_drop = ['urlKey', 'id', 'name', 'description', 'model', 'market', 'condition', 'productCategory', 'listingType', 'browseVerticals', 'favorite', 'variants']
df.drop(cols_to_drop, axis=1, inplace=True)

In [175]:
# remove rows with missing data
df.dropna(axis=0, how='any')

,title,brand,gender,media,productTraits,last sale
0,Nike Dunk Low Indigo Haze (Women's),Nike,women,{'thumbUrl': 'https://images.stockx.com/images...,"[{'name': 'Retail Price', 'value': '110'}, {'n...",130
1,Jordan 5 Retro Low Indigo Haze (Women's),Jordan,women,{'thumbUrl': 'https://images.stockx.com/images...,"[{'name': 'Retail Price', 'value': '180'}, {'n...",258
2,New Balance 550 White Natural Indigo Burgundy,New Balance,men,{'thumbUrl': 'https://images.stockx.com/images...,"[{'name': 'Retail Price', 'value': '110'}, {'n...",88
3,New Balance 550 Vintage Indigo Beige,New Balance,men,{'thumbUrl': 'https://images.stockx.com/images...,"[{'name': 'Retail Price', 'value': '110'}, {'n...",107
4,Jordan 12 Retro Indigo,Jordan,men,{'thumbUrl': 'https://images.stockx.com/images...,"[{'name': 'Retail Price', 'value': '190'}, {'n...",271
...,...,...,...,...,...,...
515,adidas Adizero Prime X Legacy Indigo,adidas,men,{'thumbUrl': 'https://images.stockx.com/images...,"[{'name': 'Retail Price', 'value': '250'}]",177
516,New Balance 327 Sky Blue Natural Indigo,New Balance,unisex,{'thumbUrl': 'https://images.stockx.com/images...,"[{'name': 'Retail Price', 'value': '100'}]",45
517,adidas Top Ten 2000 Cloud White Tech Indigo,adidas,men,{'thumbUrl': 'https://images.stockx.com/images...,"[{'name': 'Retail Price', 'value': '130'}, {'n...",150
518,adidas Ultra Boost 22 Legacy Indigo Turbo,adidas,men,{'thumbUrl': 'https://images.stockx.com/images...,"[{'name': 'Retail Price', 'value': '190'}, {'n...",65


In [181]:
# clean to extract image url and retail price
df['imageUrl'] = df['media'].apply(lambda x: (json.loads(x.replace("\'", "\"")))["thumbUrl"])
df['retailPrice'] = df['productTraits'].apply(lambda x: (json.loads(x.replace("\'", "\""))[0]["value"]))

In [185]:
df.drop(['media', 'productTraits'],axis=1)

,title,brand,gender,last sale,imageUrl,retailPrice
0,Nike Dunk Low Indigo Haze (Women's),Nike,women,130,https://images.stockx.com/images/Nike-Dunk-Low...,110
1,Jordan 5 Retro Low Indigo Haze (Women's),Jordan,women,258,https://images.stockx.com/images/Air-Jordan-5-...,180
2,New Balance 550 White Natural Indigo Burgundy,New Balance,men,88,https://images.stockx.com/images/New-Balance-5...,110
3,New Balance 550 Vintage Indigo Beige,New Balance,men,107,https://images.stockx.com/images/New-Balance-5...,110
4,Jordan 12 Retro Indigo,Jordan,men,271,https://images.stockx.com/images/Air-Jordan-12...,190
...,...,...,...,...,...,...
515,adidas Adizero Prime X Legacy Indigo,adidas,men,177,https://images.stockx.com/images/adidas-Adizer...,250
516,New Balance 327 Sky Blue Natural Indigo,New Balance,unisex,45,https://images.stockx.com/images/New-Balance-3...,100
517,adidas Top Ten 2000 Cloud White Tech Indigo,adidas,men,150,https://images.stockx.com/images/adidas-Top-Te...,130
518,adidas Ultra Boost 22 Legacy Indigo Turbo,adidas,men,65,https://images.stockx.com/images/adidas-Ultra-...,190
